In [1]:
from torch import nn
import torch.nn.functional as F
import torch
INPUT_DIM = 101 # X is 101-dimensional
HIDDEN_DIM = INPUT_DIM * 4 # Center-most latent space vector will have length of 404
NUM_CLASSES = 15 # 16 classes

print(INPUT_DIM)
print(HIDDEN_DIM)
print(NUM_CLASSES)


101
404
15


In [2]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim * 2)
        self.fc2 = nn.Linear(input_dim * 2, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, int(hidden_dim / 2))
        self.fc4 = nn.Linear(int(hidden_dim / 2), int(hidden_dim / 4))
        self.fc5 = nn.Linear(int(hidden_dim / 4), int(hidden_dim / 8))
        self.fc6 = nn.Linear(int(hidden_dim / 8), num_classes)

    def forward(self, x_in):
        z = F.relu(self.fc1(x_in))  # ReLU activation function added!
        z = F.relu(self.fc2(z))
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = F.relu(self.fc5(z))
        z = self.fc6(z)
        return z


# Initialize model
model = MLP(input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM, num_classes=NUM_CLASSES)
print(model.named_parameters)

<bound method Module.named_parameters of MLP(
  (fc1): Linear(in_features=101, out_features=202, bias=True)
  (fc2): Linear(in_features=202, out_features=404, bias=True)
  (fc3): Linear(in_features=404, out_features=202, bias=True)
  (fc4): Linear(in_features=202, out_features=101, bias=True)
  (fc5): Linear(in_features=101, out_features=50, bias=True)
  (fc6): Linear(in_features=50, out_features=15, bias=True)
)>


In [3]:
checkpoint = torch.load('model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

MLP(
  (fc1): Linear(in_features=101, out_features=202, bias=True)
  (fc2): Linear(in_features=202, out_features=404, bias=True)
  (fc3): Linear(in_features=404, out_features=202, bias=True)
  (fc4): Linear(in_features=202, out_features=101, bias=True)
  (fc5): Linear(in_features=101, out_features=50, bias=True)
  (fc6): Linear(in_features=50, out_features=15, bias=True)
)

In [3]:
""""# Inputs for inference
X_infer = pd.DataFrame([{"NOSE_X": 0.1, "NOSE_Y": 0.1, etc...}]) # all 101(or 102?) features
# Standardize
X_infer = X_scaler.transform(X_infer)
print (X_infer)
[[0.22746497 0.29242354]]
# Predict
y_infer = F.softmax(model(torch.Tensor(X_infer)), dim=1)
prob, _class = y_infer.max(dim=1)
label = label_encoder.inverse_transform(_class.detach().numpy())[0]
print (f"The probability that you have {label} is {prob.detach().numpy()[0]*100.
"""